In [2]:
import os
import sys
import random
import numpy as np
from PIL import Image, ImageFile
import cv2


__version__ = '0.3.0'


IMAGE_DIR = 'images/'

def create_mask(image_path,mask_path,save_path):
    pic_path = image_path
    show = False
    model = "hog"
    FaceMasker(pic_path, mask_path, save_path,show, model).mask()


class FaceMasker:
    KEY_FACIAL_FEATURES = ('nose_bridge', 'chin')

    def __init__(self, face_path, mask_path, save_path, show=False,model='hog'):
        self.face_path = face_path
        self.mask_path = mask_path
        self.save_path = save_path
        self.show = show
        self.model = model
        self._face_img: ImageFile = None
        self._mask_img: ImageFile = None
        self.counter = 0

    def mask(self):
        import face_recognition

        face_image_np = face_recognition.load_image_file(self.face_path)
        face_locations = face_recognition.face_locations(face_image_np, model=self.model)
        face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations)
        self._face_img = Image.fromarray(face_image_np)
        self._mask_img = Image.open(self.mask_path)

        found_face = False
        for face_landmark in face_landmarks:
            # check whether facial features meet requirement
            skip = False
            for facial_feature in self.KEY_FACIAL_FEATURES:
                if facial_feature not in face_landmark:
                    skip = True
                    break
            if skip:
                continue

            # mask face
            found_face = True
            self._mask_face(face_landmark)

        if found_face:
            if self.show:
                self._face_img.show()

            # save
            self._save()
        else:
            print('Found no face.')

    def _mask_face(self, face_landmark: dict):
        nose_bridge = face_landmark['nose_bridge']
        nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
        nose_v = np.array(nose_point)

        chin = face_landmark['chin']
        chin_len = len(chin)
        chin_bottom_point = chin[chin_len // 2]
        chin_bottom_v = np.array(chin_bottom_point)
        chin_left_point = chin[chin_len // 8]
        chin_right_point = chin[chin_len * 7 // 8]

        # split mask and resize
        width = self._mask_img.width
        height = self._mask_img.height
        width_ratio = 1.2
        new_height = int(np.linalg.norm(nose_v - chin_bottom_v))

        # left
        mask_left_img = self._mask_img.crop((0, 0, width // 2, height))
        mask_left_width = self.get_distance_from_point_to_line(chin_left_point, nose_point, chin_bottom_point)
        mask_left_width = int(mask_left_width * width_ratio)
        mask_left_img = mask_left_img.resize((mask_left_width, new_height))

        # right
        mask_right_img = self._mask_img.crop((width // 2, 0, width, height))
        mask_right_width = self.get_distance_from_point_to_line(chin_right_point, nose_point, chin_bottom_point)
        mask_right_width = int(mask_right_width * width_ratio)
        mask_right_img = mask_right_img.resize((mask_right_width, new_height))

        # merge mask
        size = (mask_left_img.width + mask_right_img.width, new_height)
        mask_img = Image.new('RGBA', size)
        mask_img.paste(mask_left_img, (0, 0), mask_left_img)
        mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

        # rotate mask
        angle = np.arctan2(chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
        rotated_mask_img = mask_img.rotate(angle, expand=True)

        # calculate mask location
        center_x = (nose_point[0] + chin_bottom_point[0]) // 2
        center_y = (nose_point[1] + chin_bottom_point[1]) // 2

        offset = mask_img.width // 2 - mask_left_img.width
        radian = angle * np.pi / 180
        box_x = center_x + int(offset * np.cos(radian)) - rotated_mask_img.width // 2
        box_y = center_y + int(offset * np.sin(radian)) - rotated_mask_img.height // 2

        # add mask
        self._face_img.paste(mask_img, (box_x, box_y), mask_img)

    def _save(self):
        path_splits = os.path.splitext(self.face_path)
         #This line may cause error given path so adjust according to you
        filename = self.face_path.split("\\")[-1]
        new_face_path = self.save_path + '\with-mask-'+ filename
        self._face_img.save(new_face_path)
        print(f'Save to {new_face_path}')
       

    @staticmethod
    def get_distance_from_point_to_line(point, line_point1, line_point2):
        distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                          (line_point1[0] - line_point2[0]) * point[1] +
                          (line_point2[0] - line_point1[0]) * line_point1[1] +
                          (line_point1[1] - line_point2[1]) * line_point1[0]) / \
                   np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                           (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
        return int(distance)

In [6]:
import random
#mask_img_names = ["n-95-1.png","n-95-2.png","n-95-3.png","n-95-4.png","n-95-5.png"]
#mask_img_names = ["three_layer-1.png","three_layer-2.png","three_layer-3.png","three_layer-4.png","three_layer-5.png"]
mask_img_names =["hanker-1.png","hanker-2.png","hanker-3.png"]
for filename in os.listdir(os.path.join(os.getcwd(),"data/")):
    image_path = os.path.join(os.getcwd(),"data",filename)
    save_path = os.path.join(os.getcwd(),"hankerchief") 
    mask_img_choice = random.choice(mask_img_names)
    mask_path = os.path.join(os.getcwd(),"images",mask_img_choice)
    create_mask(image_path,mask_path,save_path)


Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-0.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-1.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-10.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-100.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-101.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-102.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-104.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-105.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-106.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-107.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-108.jpg
Save

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-192.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-193.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-194.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-195.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-196.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-197.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-198.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-2.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-20.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-201.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-203.jpg
Sa

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-293.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-294.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-295.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-297.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-298.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-299.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-3.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-30.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-300.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-301.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-302.jpg
Sa

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-397.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-398.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-399.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-4.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-40.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-400.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-401.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-402.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-403.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-404.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-405.jpg
Sa

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-57.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-58.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-59.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-6.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-61.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-62.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-63.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-64.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-65.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-66.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-67.jpg
Save to C:\

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_145.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_147.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_148.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_149.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_150.jpg
Found no face.
Found no face.
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_153.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_154.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_155.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_156.jpg
Save to

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_227.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_229.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_23.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_230.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_231.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_232.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_233.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_234.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_235.jpg
Save to C:\Users\deadl\Downloads\covid

Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_314.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_315.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_316.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_317.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_318.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_319.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_32.jpg
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_33.jpg
Found no face.
Save to C:\Users\deadl\Downloads\covid 19\Mask Recognization\hankerchief\with-mask-augmented_image_36.jpg
Save to C:\Users\deadl\Do

['C:',
 'Users',
 'deadl',
 'Downloads',
 'covid 19',
 'Mask Recognization',
 'data',
 'augmented_image_319.jpg']